# Payoff d'un call spread (bull call) sur SPY

TP dédié au payoff d'un call spread vertical (long K1, short K2) sur SPY.
Objectifs :
1. Fixer un spot de référence via l'historique SPY.
2. Construire un spread haussier : achat d'un call ITM/ATM (K1), vente d'un call OTM (K2 > K1).
3. Visualiser la courbe de payoff bornée entre 0 et K2-K1.

In [ ]:
%pip -q install yfinance matplotlib pandas numpy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("seaborn-v0_8-darkgrid")

def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_call_spread(spot: float, k_long: float, k_short: float) -> float:
    return payoff_call(spot, k_long) - payoff_call(spot, k_short)

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
k_long = spot_ref * 0.98
k_short = spot_ref * 1.05
spot_ref, k_long, k_short

## Évolution du sous-jacent (SPY)
Clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff du call spread long
Spread borné : gain maximum égal à K2 - K1, perte limitée à la prime nette.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_call_spread(S, k_long, k_short) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff call spread long")
ax.axvline(k_long, color="gray", linestyle=":", label=f"K long = {k_long:.2f}")
ax.axvline(k_short, color="gray", linestyle="--", label=f"K short = {k_short:.2f}")
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Bull call spread sur SPY : payoff vs spot")
ax.legend()
plt.show()